In [5]:
import os
import csv

import gzip
import shutil

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
import glob

In [3]:
#working with sample_main.fits as a test
homedir = os.getenv("HOME")
vf = Table.read(homedir+'/github/research/sample_main.fits')

dec = vf['DEC']
ra = vf['RA']
size = vf['radius']    #arcsec
name = vf['prefix']

In [10]:
#example


dec_ex = dec[0]
ra_ex = ra[0]
size_ex = size[0]
name_ex = name[0]

#the following yields a .csv file containing allwise four (w1-w4) data rows for the given coords.
irsa_call = '\"https://irsa.ipac.caltech.edu/ibe/search/wise/allwise/p3am_cdd?POS=' + str(ra_ex) + ',' + str(dec_ex) + '&ct=csv&mcen\"'
filename = homedir+'/github' + '/allwise.csv'
if os.path.exists(filename):
    os.remove(filename)

#copies .csv to pathname (i.e., filename)
os.system("curl -o %s %s" % (filename, irsa_call))

#this .csv information is then extractable to acquire the desired .fits cutouts, like so:
outdir = homedir+'/github/allwise_cutouts/'

with open(filename) as csvfile:
    reader = csv.DictReader(csvfile)
    for line in reader:
        band = line['band']
        if int(band) != 3:
            pass
        else:
            coadd_id = line['coadd_id']
            coaddgrp = coadd_id[:2]
            coadd_ra = coadd_id[:4]
            #int for intensity, unc for uncertainty; only pulls w3 files
            filename_int = coadd_id + '-w3'+'-int-3.fits.gz'
            filename_unc = coadd_id + '-w3'+'-unc-3.fits.gz'
            #url dongxi
            irsa_url = "%s/%s/%s/" % (coaddgrp, coadd_ra, coadd_id)
            irsa_url_int = irsa_url + filename_int
            irsa_url_unc = irsa_url + filename_unc
            irsa_url_int = '\"https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/' + irsa_url_int +'?center=' + str(ra_ex) + ',' + str(dec_ex) +'&size=' + str(size_ex*2) + 'arcsec' + '\"'
            irsa_url_unc = '\"https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/' + irsa_url_unc +'?center=' + str(ra_ex) + ',' + str(dec_ex) +'&size=' + str(size_ex*2) + 'arcsec' + '\"'
            filename_int = outdir + filename_int
            filename_unc = outdir + filename_unc
            #grab int .gz
            os.system("curl -o %s %s" % (filename_int, irsa_url_int))
            #rename file
            name_desired_int = outdir + str(name_ex) + '-int.fits.gz'
            os.rename(filename_int, name_desired_int)
            #grab unc .gz
            os.system("curl -o %s %s" % (filename_unc, irsa_url_unc))
            #rename file
            name_desired_unc = outdir + str(name_ex) + '-unc.fits.gz'
            os.rename(filename_unc, name_desired_unc)

In [13]:
#decompress resulting gzip files (from Stackoverflow)

#int, name_desired_int, name_desired_int_nogz
name_desired_int_nogz = outdir + str(name_ex) + '-int.fits'
with gzip.open(name_desired_int,'rb') as f_in:
    with open(name_desired_int_nogz,'wb') as f_out:
        shutil.copyfileobj(f_in,f_out)
        
#unc, name_desired_unc, name_desired_unc_nogz
name_desired_unc_nogz = outdir + str(name_ex) + '-unc.fits'
with gzip.open(name_desired_unc,'rb') as f_in:
    with open(name_desired_unc_nogz,'wb') as f_out:
        shutil.copyfileobj(f_in,f_out)

### Näschste zeit:
### 1. learn how to unzip .gz files using a python command $\checkmark$
### 2. create 'loop' function using the above, perhaps incorporate into rungalfit.py script (both local and remote versions) $\checkmark$

In [16]:
#lastly, the function itself. I added the above to my run_galfit.py routine, so the remaining task 
#entails a good ol' test.

In [17]:
%run ~/github/research/run_galfit.py

In [20]:
galaxy_sample = vf[0:10]
for n in range(0,len(galaxy_sample)):
    g = galaxy(galaxy_sample['RA'][n], galaxy_sample['DEC'][n],
                      galaxy_sample['radius'][n], name = galaxy_sample['prefix'][n], band='3')
    g.get_allwise_image()
#the one qualm I possess relates to the extraneous .txt files that appear in the galfit/research directory,
#which I can simply manually delete if I am only wanting the allwise .fits cutouts. otherwise, I should be sure
#that I change directories accordingly so ALL GALFIT output is ushered into the desired spot.

-------
## Scrap.

Example: Search for ALLWISE images within 10 arcseconds of ra,dec = (19.005875, -5.084472). Return the result as in comma-separated value format.

wget -O example1.csv "https://irsa.ipac.caltech.edu/SIA?COLLECTION=wise_allwise&POS=circle+-164.7+-5.8+0.002777&RESPONSEFORMAT=CSV"

This will get you a CSV file listing the metadata for all the images that match your search criteria.

One of the columns will be “access_url”, which will have values like: https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/19/1959/1959m061_ac51/1959m061_ac51-w3-cov-3.fits.gz

Then, if you want a cutout of that images, append the “center” and “size” parameters, so you have something like this: https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/03/0390/0390p605_ac51/0390p605_ac51-w1-int-3.fits?center=39.000655,60.577778&size=200pix

Another tip: You can make the search position radius = 0. 

The resulting CSV list will include coverage, intensity, and uncertainty images for each WISE band. You can either parse the CSV file to get the images you want, or you can set the BAND. In any case, you will have to parse the file to find the URL of the intensity map versus the coverage map.